In [1]:
# 用户输入句子
inputStr = input("请输入句子：")


In [2]:
inputStr

'周新钢是哪个案件的当事人？'

In [1]:
# inputStr2='荣成市人民法院审理的追偿权纠纷案有多少件?山东省高级人民法院是否审理过张开祥相关的案件?'

In [20]:
inputStr2 = '周新钢是哪个案件的当事人？'


In [21]:
def text_to_bmes(input_text, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for char in input_text:
            if char == '\n':
                f.write('\n')
            else:
                f.write(f'{char} O\n')
        f.write('\n')


input_text = inputStr2

output_file = "./ResumeNER/predict.char.bmes"

text_to_bmes(input_text, output_file)


In [22]:
import pickle
import sys
sys.path.append("..")  # 将上级目录添加到Python路径
from utils import load_model, extend_maps, prepocess_data_for_lstmcrf
from data import build_corpus,build_map
from evaluating import Metrics
from evaluate import ensemble_evaluate


In [23]:
from os.path import join
from codecs import open


In [26]:
predict_word_lists, predict_tag_lists = build_corpus("predict", make_vocab=False,data_dir="./ResumeNER")

Provided split value: predict


ValueError: not enough values to unpack (expected 2, got 0)

In [6]:
train_word_lists, train_tag_lists, word2id, tag2id = build_corpus("train",data_dir="./ResumeNER")

Provided split value: train


In [7]:
crf_word2id, crf_tag2id = extend_maps(word2id, tag2id, for_crf=True)

In [8]:
bilstm_model = load_model('./ckpts/bilstm_crf.pkl')


In [9]:
bilstm_model.model.bilstm.bilstm.flatten_parameters()  # remove warning


In [10]:
predict_word_lists, predict_tag_lists = prepocess_data_for_lstmcrf(
        predict_word_lists, predict_tag_lists, test=True
    )

In [11]:
lstmcrf_pred, target_tag_list = bilstm_model.test(predict_word_lists, predict_tag_lists,
                                                  crf_word2id, crf_tag2id)


In [13]:
lstmcrf_pred


[['B-NAME', 'M-NAME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]

In [14]:
input_text


'周新钢是哪个案件的当事人？'

In [15]:
simplified_tags = [tag.split('-')[-1] for tag in lstmcrf_pred[0]]

result_dict = {word: tag for word, tag in zip(input_text, simplified_tags)}

print(result_dict)


{'周': 'NAME', '新': 'NAME', '钢': 'O', '是': 'O', '哪': 'O', '个': 'O', '案': 'O', '件': 'O', '的': 'O', '当': 'O', '事': 'O', '人': 'O', '？': 'O'}


In [16]:
result_dict = {}
current_name = []

for word, tag in zip(input_text, simplified_tags):
    if tag == "NAME":
        current_name.append(word)
    elif current_name:
        name_str = ''.join(current_name)
        if name_str not in result_dict:
            result_dict[name_str] = "NAME"
        current_name = []

print(result_dict)


{'周新': 'NAME'}


In [17]:
tag2id

{'B-NAME': 0,
 'E-NAME': 1,
 'O': 2,
 'B-CONT': 3,
 'M-CONT': 4,
 'E-CONT': 5,
 'B-RACE': 6,
 'E-RACE': 7,
 'B-TITLE': 8,
 'M-TITLE': 9,
 'E-TITLE': 10,
 'B-EDU': 11,
 'M-EDU': 12,
 'E-EDU': 13,
 'B-ORG': 14,
 'M-ORG': 15,
 'E-ORG': 16,
 'M-NAME': 17,
 'B-PRO': 18,
 'M-PRO': 19,
 'E-PRO': 20,
 'S-RACE': 21,
 'S-NAME': 22,
 'B-LOC': 23,
 'M-LOC': 24,
 'E-LOC': 25,
 'M-RACE': 26,
 'S-ORG': 27,
 '<unk>': 28,
 '<pad>': 29,
 '<start>': 30,
 '<end>': 31}

In [18]:
REMOVE_O = False

In [19]:
metrics = Metrics(target_tag_list, lstmcrf_pred, remove_O=REMOVE_O)
metrics.report_scores()
metrics.report_confusion_matrix()


           precision    recall  f1-score   support
        O     1.0000    0.8462    0.9167        13
avg/total     1.0000    0.8462    0.9167        13

Confusion Matrix:
              O 
      O      11 
